In [483]:
# @title Upload CSV
import pandas as pd
import random

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [484]:
# @title Gabungkan data_dosen dengan dosen
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [485]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    for id_counter, (hari, ruang, jam) in enumerate(
        [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
    ):
        wolf.append({
            "id_slot": id_counter,
            "mata_kuliah": None,
            "dosen": None,
            "ruang": ruang,
            "hari": hari,
            "jam_mulai": jam['jam_awal'],
            "jam_selesai": jam['jam_akhir'],
            "kelas": None,
            "sks": None,
            "metode": None
        })
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [486]:
# @title Inisialisasi Populasi (GWO)
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        return datetime.strptime(t, "%H:%M:%S").hour * 60 + datetime.strptime(t, "%H:%M:%S").minute
    except ValueError:
        return datetime.strptime(t, "%H:%M").hour * 60 + datetime.strptime(t, "%H:%M").minute

def create_random_schedule():
    slots = slot_generator()
    merged_shuffled = merged_df.sample(frac=1).iterrows()
    
    # Struktur tracking untuk hard constraints
    room_schedule = defaultdict(list)
    teacher_schedule = defaultdict(list)

    for index, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        possible_positions = list(range(len(slots) - sks + 1))
        random.shuffle(possible_positions)

        placed = False
        for i in possible_positions:
            block = slots[i:i+sks]
            
            # Validasi dasar slot
            all_empty = all(slot['mata_kuliah'] is None for slot in block)
            same_hari = len({slot['hari'] for slot in block}) == 1
            same_ruang = (metode == 'Online') or (len({slot['ruang'] for slot in block}) == 1)
            
            if not (all_empty and same_ruang and same_hari):
                continue
                
            # Ekstrak informasi waktu
            target_hari = block[0]['hari']
            target_ruang = block[0]['ruang'] if metode != 'Online' else None
            block_start = time_to_minutes(block[0]['jam_mulai'])
            block_end = time_to_minutes(block[-1]['jam_selesai'])
            
            # Hard constraint 1: Cek konflik ruang
            room_conflict = False
            if metode != 'Online':
                for existing_start, existing_end in room_schedule.get((target_ruang, target_hari), []):
                    if not (block_end <= existing_start or block_start >= existing_end):
                        room_conflict = True
                        break
            # Hard constraint 2: Cek konflik dosen
            teacher_conflict = False
            for existing_start, existing_end in teacher_schedule.get((dosen, target_hari), []):
                if not (block_end <= existing_start or block_start >= existing_end):
                    teacher_conflict = True
                    break
            
            if not room_conflict and not teacher_conflict:
                # Update data slot
                ruang = 'Online' if metode == 'Online' else target_ruang
                for slot in block:
                    slot.update({
                        "mata_kuliah": mata_kuliah,
                        "dosen": dosen,
                        "kelas": kelas,
                        "sks": sks,
                        "metode": metode,
                        "ruang": ruang
                    })
                # Update tracking
                if metode != 'Online':
                    room_schedule[(target_ruang, target_hari)].append((block_start, block_end))
                teacher_schedule[(dosen, target_hari)].append((block_start, block_end))
                placed = True
                break
                
                # Fallback untuk soft constraint jika tidak ditemukan slot
        if not placed:
            # Implementasi fallback untuk soft constraint di sini
            pass
    return slots

population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

for i in range(population_size):
    print(f"\nIndividu {i+1}:")
    for slot in population[i][:5]:
    # for slot in population[i]:
        print(slot)


Individu 1:
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': 'Aljabar Linear Matrik', 'dosen': 'Guntur  Maulana Zamroni, B.Sc., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': 'A', 'sks': 2, 'metode': 'Offline'}
{'id_slot': 5, 'mata_kuliah': 'Aljabar Linear Matrik', 'dosen': 'Guntur  Maulana Zamroni, B.Sc., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'k

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
jadwal dosen mengajar kelas yang berbeda pada jam yang sama
jadwal ruang kelas ruang kelas digunakan lebih dari 1 dosen 

Konstrain ringan:
dosen mengajar pada hari yang sama
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [487]:
# @title Fitness Function (GWO)
from collections import defaultdict

def calculate_fitness(schedule):
    max_score = 100
    penalty = 0
    
    # Struktur tracking untuk semua konstrain
    teacher_schedule = defaultdict(list)
    room_schedule = defaultdict(list)
    teacher_daily_courses = defaultdict(set)
    class_time_bounds = defaultdict(lambda: {'min': float('inf'), 'max': -float('inf')})
    penalized_classes = set()

    # Fungsi untuk menghitung overlap
    def count_overlaps(intervals):
        intervals.sort()
        overlaps = 0
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                overlaps += 1
        return overlaps

    # First pass: Kumpulkan semua data
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
            
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        hari = slot['hari']
        dosen = slot['dosen']
        ruang = slot['ruang']
        kelas = slot['kelas']
        mk = slot['mata_kuliah']

        # Track untuk hard constraints
        teacher_schedule[(dosen, hari)].append((start, end))
        if ruang != 'Online':
            room_schedule[(ruang, hari)].append((start, end))

        # Track untuk soft constraints
        teacher_daily_courses[(dosen, hari)].add(mk)
        class_time_bounds[(kelas, hari)]['min'] = min(class_time_bounds[(kelas, hari)]['min'], start)
        class_time_bounds[(kelas, hari)]['max'] = max(class_time_bounds[(kelas, hari)]['max'], start)

    # Hitung hard constraints
    # 1. Dosen mengajar di kelas berbeda pada jam yang sama
    for intervals in teacher_schedule.values():
        penalty += count_overlaps(intervals) * 1  # Berat 1 per konflik
        
    # 2. Ruang digunakan oleh lebih dari 1 kelas di waktu sama
    for intervals in room_schedule.values():
        penalty += count_overlaps(intervals) * 1  # Berat 1 per konflik

    # Hitung soft constraints
    # 1. Dosen mengajar di hari yang sama
    for courses in teacher_daily_courses.values():
        if len(courses) > 1:
            penalty += 0.5 * (len(courses) - 1)  # Ringan 0.5 per tambahan MK
            
    # 2. Kelas di jam pertama/terakhir
    for (kelas, hari), bounds in class_time_bounds.items():
        if (kelas, hari) not in penalized_classes:
            min_time = bounds['min']
            max_time = bounds['max']
            
            # Cek keberadaan slot di jam awal/akhir
            has_min = any(
                time_to_minutes(slot['jam_mulai']) == min_time 
                for slot in schedule 
                if slot['kelas'] == kelas 
                and slot['hari'] == hari 
                and slot['mata_kuliah']
            )
            
            has_max = any(
                time_to_minutes(slot['jam_mulai']) == max_time 
                for slot in schedule 
                if slot['kelas'] == kelas 
                and slot['hari'] == hari 
                and slot['mata_kuliah']
            )
            
            if has_min:
                penalty += 0.5
            if has_max:
                penalty += 0.5
                
            penalized_classes.add((kelas, hari))

    return {
        'score': max(0, max_score - penalty),  # Pastikan tidak negatif
        'penalty': penalty,
        'violations': {
            'hard_teacher': sum(count_overlaps(v) for v in teacher_schedule.values()),
            'hard_room': sum(count_overlaps(v) for v in room_schedule.values()),
            'soft_teaching_day': sum(len(v)-1 for v in teacher_daily_courses.values() if len(v) > 1),
            'soft_edge_time': sum(1 for _ in penalized_classes)
        }
    }

# Generate populasi
population_size = 100
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    print(f"Fitness: {fitness['score']}")
    print(f"Total Penalty: {fitness['penalty']}")
    print("Detail Pelanggaran:")
    print(f"- Konflik Dosen: {fitness['violations']['hard_teacher']}")
    print(f"- Konflik Ruang: {fitness['violations']['hard_room']}")
    print(f"- Dosen mengajar lebih dari 1 MK per hari: {fitness['violations']['soft_teaching_day']}")
    print(f"- Kelas Jam Ujung: {fitness['violations']['soft_edge_time']}")


Individu 1:
Fitness: 58.0
Total Penalty: 42.0
Detail Pelanggaran:
- Konflik Dosen: 4
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 14
- Kelas Jam Ujung: 31

Individu 2:
Fitness: 60.0
Total Penalty: 40.0
Detail Pelanggaran:
- Konflik Dosen: 0
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 14
- Kelas Jam Ujung: 33

Individu 3:
Fitness: 61.0
Total Penalty: 39.0
Detail Pelanggaran:
- Konflik Dosen: 0
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 12
- Kelas Jam Ujung: 33

Individu 4:
Fitness: 57.0
Total Penalty: 43.0
Detail Pelanggaran:
- Konflik Dosen: 1
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 14
- Kelas Jam Ujung: 35

Individu 5:
Fitness: 58.0
Total Penalty: 42.0
Detail Pelanggaran:
- Konflik Dosen: 1
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 MK per hari: 16
- Kelas Jam Ujung: 33

Individu 6:
Fitness: 56.0
Total Penalty: 44.0
Detail Pelanggaran:
- Konflik Dosen: 1
- Konflik Ruang: 0
- Dosen mengajar lebih dari 1 

In [488]:
# @title Postprocessing Return solusi terbaik (Alpha).
max_iter = 10

def get_best_wolves(population):
    fitness_scores = [(schedule, calculate_fitness(schedule)) for schedule in population]
    sorted_wolves = sorted(fitness_scores, key=lambda x: x[1], reverse=True)
    return sorted_wolves[0][0], sorted_wolves[1][0], sorted_wolves[2][0]

for iterasi in range(max_iter):
    alpha, beta, delta = get_best_wolves(population)
    new_population = []

    for wolf in population:
        if wolf in [alpha, beta, delta]:
            new_population.append(wolf)
            continue
        new_wolf = copy.deepcopy(wolf)
        for slot in new_wolf:
            alpha_slot = next((s for s in alpha if s["id_slot"] == slot["id_slot"]), None)
            beta_slot = next((s for s in beta if s["id_slot"] == slot["id_slot"]), None)
            delta_slot = next((s for s in delta if s["id_slot"] == slot["id_slot"]), None)
            if alpha_slot and beta_slot and delta_slot:
                choices = [alpha_slot, beta_slot, delta_slot]
                best_choice = sorted(choices, key=lambda x: x["id_slot"])[0]
                slot.update({"mata_kuliah": best_choice["mata_kuliah"], "dosen": best_choice["dosen"], "kelas": best_choice["kelas"]})
        new_population.append(new_wolf)
    population = new_population
    print(f"Iterasi {iterasi+1} - Best Fitness: {calculate_fitness(alpha)}")

final_schedule = get_best_wolves(population)[0]
for slot in sorted(final_schedule, key=lambda x: x["id_slot"]):
    print(slot)

TypeError: '<' not supported between instances of 'dict' and 'dict'